In [1]:
import os
import re
import nltk
import gensim
import zipfile
from pymystem3 import Mystem
import pandas as pd
from lxml import html
import numpy as np
from collections import defaultdict, Counter
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from string import punctuation
punctuation += '«»—…“”*№–'
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('russian'))
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ира\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def normalize(text):
    words = [word.strip(punctuation) 
        for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form
            for word in words if word
            and word not in stops]
    return ' '.join(words)

def tokenize(text):
    words = [word.strip(punctuation) 
        for word in text.lower().split()]
    return ' '.join(words)

*Возьмите данные соревнования по определению перефразирования - http://paraphraser.ru/download/get?file_id=1*

In [5]:
corpus_xml = html.fromstring(
    open('paraphraser/paraphrases.xml', 
         'rb').read())

In [6]:
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])

data = pd.DataFrame({
    'text_1':texts_1,
    'text_2':texts_2,
    'label':classes})

In [7]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [8]:
data.head(2)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...


*Задание делится на 2 части:*

*1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (любой). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше.* 

*Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие.*

## Часть 1. Сравнение модели, обученной самостоятельно, и модели с русвекторес

### Модель, обученная самостоятельно (на корпусе из кругосвета)

Загружаем и нормализуем корпус

In [4]:
fpath = 'corpus_hum.txt'
with open(fpath, 'r', encoding='utf-8') as f:
    corpus_norm = [normalize(text) for text in f.readlines()]
corpus_norm = [text for text in corpus_norm if text]
corpus_norm[:2]    

['абай василий васо иван 1900–2001 русский лингвист родиться 2 15 декабрь 1900 с.коби тифлисский губерния ныне грузия 1925 окончить факультет общественный наука ленинградский университет 1928 аспирантура 1928–1930 сотрудник кавказский историко-археологический институт ан ссср 1930 полвека работать яфетический институт затем институт язык мышление институт языкознание ан ссср ленинград 1950 москва доктор филологический наука 1962 профессор 1969 лауреат государственный премия ссср 1981 почётный член азиатский королевский общество великобритания ирландия 1966 член-корреспондент финно-угорский общество хельсинки 1973 умереть абай москва 18 март 2001',
 'также тема']

Обучаем ворд2век

In [12]:
%%time
w2v_model = gensim.models.Word2Vec([text.split() 
                             for text
                             in corpus_norm],
                             size=50,
                             sg=1)

Wall time: 1min 52s


In [5]:
# функция из семинара которая эмбеддит текст
def get_embedding(text, model):
    dim=model.vector_size
    text = text.split()
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i, word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word] #счетчик слова
                           /total)
        except (KeyError, ValueError):
            continue
#     проверяем, есть ли вообще ненулевые
#     значения в этой матрице
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

Эмбеддим пары текстов

In [70]:
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v_model)
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v_model)

Склеиваем

In [71]:
X_text_w2v = np.concatenate([X_text_1_w2v,
                            X_text_2_w2v], axis=1)

Обучаем рэндом форест

In [103]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=7,
                            min_samples_leaf=15,
                            class_weight='balanced')
clf.fit(X_text_w2v, data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

*ВАЖНО: Оценивать модели нужно с помощью кросс-валидации! Метрика - f1.*

Оцениваем кросс-валидацией

In [107]:
for score_type in 'weighted micro macro'.split():
    print ('Mean validation {} F1-score:'.format(score_type), 
        round(
            float(np.mean(cross_val_score(
            estimator=clf,
            X=X_text_w2v,
            y=data['label'],   
            scoring='f1_{}'.format(score_type),
            cv=5, 
            n_jobs=-1))),
            3))

Mean validation weighted F1-score: 0.454
Mean validation micro F1-score: 0.447
Mean validation macro F1-score: 0.439


## Модель с русвекторес

Модель скачали заранее. Это скипграм, обученный на тайге.

In [108]:
with zipfile.ZipFile('185.zip', 'r') as archive:
    stream = archive.open('model.bin')
    rv_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Код из семинара и из туториала русвекторес

In [156]:
mapping = {}
mapping_file='ru-rnc.map.txt'
with open(mapping_file, 'r', encoding='utf-8') as f:
    for pair in f.readlines():
        pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
#         if len(pair) > 1:
        mapping[pair[0]] = pair[1]
# print(mapping)

m = Mystem()
def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)

    return ' '.join(tokens)

In [152]:
data.columns

Index(['text_1', 'text_2', 'label', 'text_1_norm', 'text_2_norm'], dtype='object')

Вообще эта функция нормализует

In [157]:
normalize_mystem(text='Текст нужно передать функции в виде строки!')

'текст_NOUN нужно_ADV передавать_VERB функция_NOUN в_ADP вид_NOUN строка_NOUN'

Но применять мы её, конечно, не будем (это очень долго). Спасибо, что применили за нас.

In [160]:
# data['text_1_upos'] = data['text_1'].apply(normalize_mystem)
# data['text_2_upos'] = data['text_2'].apply(normalize_mystem)
data_upos = pd.read_csv('data_paraphraser_norm.csv')
data['text_1_upos'] = data_upos['text_1_norm']
data['text_2_upos'] = data_upos['text_2_norm']

Теперь делаем то же самое для предобученной кем-то модели

In [266]:
X_text_1_w2v = np.zeros((len(data['text_1_upos']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_upos']), dim))

for i, text in enumerate(data['text_1_upos'].values):
    X_text_1_w2v[i] = get_embedding(text, rv_model)
for i, text in enumerate(data['text_2_upos'].values):
    X_text_2_w2v[i] = get_embedding(text, rv_model)

In [267]:
X_text_w2v = np.concatenate([X_text_1_w2v,
                            X_text_2_w2v], axis=1)

In [268]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=7,
                            min_samples_leaf=15,
                            class_weight='balanced')
clf.fit(X_text_w2v, data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [269]:
for score_type in 'weighted micro macro'.split():
    print ('Mean validation {} F1-score:'.format(score_type), 
        round(
            float(np.mean(cross_val_score(
            estimator=clf,
            X=X_text_w2v,
            y=data['label'],   
            scoring='f1_{}'.format(score_type),
            cv=5, 
            n_jobs=-1))),
            3))

Mean validation weighted F1-score: 0.45
Mean validation micro F1-score: 0.449
Mean validation macro F1-score: 0.431


Кругосветный w2v:

Mean validation weighted F1-score: 0.454

Mean validation micro F1-score: 0.447

Mean validation macro F1-score: 0.439

Если сравнивать с моделью на кругосвете, разницы не видно вообще. Видимо потому что обе эти модели взяты с потолка и не учитывают специфику текстов парафразов.

## Часть 2. Обучение модели на косинусных близостях пар текстов

*2) Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары).*

In [182]:
%%time
# наш w2v
w2v_embeddings = list(zip([get_embedding(text, w2v_model) 
                  for text in data['text_1_norm'].values], 
                          [get_embedding(text, w2v_model) 
                  for text in data['text_2_norm'].values]))
data['w2v_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(w2v_embeddings[i][0]), 
    np.atleast_2d(w2v_embeddings[i][1]))[0,0] for i in range(len(data))]

Wall time: 3.24 s


Обучим на кругосвете фасттекст

In [17]:
%%time
ft_model = gensim.models.FastText([text.split()
                                for text 
                                in corpus_norm],
                                size=50,
# Minimum length of char n-grams 
# to be used for training word representations
                                min_n=4,
# Max length of char ngrams 
                                max_n=8)

Wall time: 4min 47s


In [188]:
%%time
# fasttext
ft_embeddings = list(zip([get_embedding(text, ft_model) 
                  for text in data['text_1_norm'].values], 
                          [get_embedding(text, ft_model) 
                  for text in data['text_2_norm'].values]))
data['ft_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(ft_embeddings[i][0]), 
    np.atleast_2d(ft_embeddings[i][1]))[0,0] for i in range(len(data))]

Wall time: 4min 12s


In [194]:
%%time
# skipgram с rusvectores
rv_embeddings = list(zip([get_embedding(text, rv_model) 
                  for text in data['text_1_upos'].values], 
                          [get_embedding(text, rv_model) 
                  for text in data['text_2_upos'].values]))
data['rv_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(rv_embeddings[i][0]), 
    np.atleast_2d(rv_embeddings[i][1]))[0,0] for i in range(len(data))]

Wall time: 4.39 s


Изначально в словаре векторайзера было 1000 фич, но NMF очень медленно обучался. Так что SVD обучили на 1000 фич, а NMF кое-как на 200 (зато за полчаса, а не за сто лет).

In [226]:
tv = TfidfVectorizer(min_df=3, max_df=0.4,
#                     max_features=1000
                    max_features=200)
tv.fit(pd.concat([data['text_1_norm'],
                 data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=200,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [205]:
svd = TruncatedSVD(200)

data['svd_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(svd.fit_transform(
        tv.transform(data['text_1_norm']))[i]), 
    np.atleast_2d(svd.fit_transform(
        tv.transform(data['text_2_norm']))[i]))[0,0] 
                                 for i in range(len(data))]

На этом моменте мы пошли в колаб, но это не принесло результата, пришлось просто обучить из рук вон плохо.

In [3]:
# data.to_csv('5cosine_similarities.csv', encoding='utf-8')
data = pd.read_csv('5cosine_similarities.csv')

In [231]:
nmf = NMF(n_components=3, 
#           ‘random’ | ‘nndsvd’ | ‘nndsvda’
#           init='nndsvdar',
          init=None,
          solver='cd', 
          beta_loss='frobenius', 
#           Tolerance of the stopping condition
#           tol=0.0001,
          tol=0.1, 
#           max_iter=200,
          max_iter=100, 
          random_state=1, 
#           Constant that multiplies the regularization terms
          alpha=0.0, 
#           For l1_ratio = 0 the penalty is 
#           an elementwise L2 penalty (aka Frobenius Norm). 
          l1_ratio=0.0, 
#           verbose=1, 
#           If true, randomize the order of coordinates in the CD solver.
          shuffle=False)

In [271]:
# %%time Wall time: 34min 48s

data['nmf_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(nmf.fit_transform(
        tv.transform(data['text_1_norm']))[i]), 
    np.atleast_2d(nmf.fit_transform(
        tv.transform(data['text_2_norm']))[i]))[0,0] 
                                 for i in range(len(data))]

In [240]:
# она настолько плохая, что скорее посчитает парафразами всё кроме парафразов
# вообще f1-micro отдельно этой модели около 30
print(len(data[data['nmf_cosine_similarity']==1]))
print(len(data[(data['nmf_cosine_similarity']==1) & (data['label']=='1')]))
print(len(data[(data['nmf_cosine_similarity']==1) & (data['label']=='0')]))
print(len(data[(data['nmf_cosine_similarity']==1) & (data['label']=='-1')]))

495
105
228
162


*Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).*

*SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части).*

Строим обучающую выборку

In [248]:
X5 = data[data.columns[7:]].to_numpy()

Обучаем рэндом форест на близостях

In [249]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=7,
                            min_samples_leaf=15,
                            class_weight='balanced')
clf.fit(X5, data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Оцениваемся на кросс-валидации

In [254]:
%%time

for score_type in 'weighted micro macro'.split():
    print ('Mean validation {} F1-score:'.format(score_type), 
        round(
            float(np.mean(cross_val_score(
            estimator=clf,
#             X=train_X,
            X=X5,
#             y=train_y,
            y=data['label'],   
            scoring='f1_{}'.format(score_type),
            cv=15, 
            n_jobs=-1))),
            5))

# Результаты на кругосветном w2v:
# Mean validation weighted F1-score: 0.454
# Mean validation micro F1-score: 0.447
# Mean validation macro F1-score: 0.439

Mean validation weighted F1-score: 0.54521
Mean validation micro F1-score: 0.54904
Mean validation macro F1-score: 0.54766
Wall time: 22.4 s


**На близостях получилось лучше, ура! Причём даже самая плохая модель, как кажется, внесла свой вклад (без неё, то есть на 4 близостях вместо 5, метрика была чуть хуже).**

## Всё ещё часть 2

*Попробуйте улучшить метрику, изменив параметры в методах векторизации.*

In [6]:
%%time
# поменяли размер векторов
w2v_model = gensim.models.Word2Vec([text.split() 
                             for text
                             in corpus_norm],
                             size=100,
                             sg=1)

Wall time: 2min 11s


In [7]:
%%time
# поменяли размер векторов и увеличили длину нграмов
ft_model = gensim.models.FastText([text.split()
                                for text 
                                in corpus_norm],
                                size=100,
# Minimum length of char n-grams 
# to be used for training word representations
                                min_n=5,
# Max length of char ngrams 
                                max_n=9)

Wall time: 4min 29s


In [8]:
%%time
# fasttext
ft_embeddings = list(zip([get_embedding(text, ft_model) 
                  for text in data['text_1_norm'].values], 
                          [get_embedding(text, ft_model) 
                  for text in data['text_2_norm'].values]))
data['ft_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(ft_embeddings[i][0]), 
    np.atleast_2d(ft_embeddings[i][1]))[0,0] for i in range(len(data))]

c:\users\ира\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


Wall time: 7.49 s


In [10]:
# поменяли количество фичей
tv = TfidfVectorizer(min_df=3, max_df=0.4,
#                     max_features=1000
                    max_features=400)
tv.fit(pd.concat([data['text_1_norm'],
                 data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=400,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
# поменяли размер векторов
svd = TruncatedSVD(300)

data['svd_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(svd.fit_transform(
        tv.transform(data['text_1_norm']))[i]), 
    np.atleast_2d(svd.fit_transform(
        tv.transform(data['text_2_norm']))[i]))[0,0] 
                                 for i in range(len(data))]

In [12]:
nmf = NMF(n_components=10, #поменяли
#           ‘random’ | ‘nndsvd’ | ‘nndsvda’
          init='nndsvdar', #поменяли
#           init=None,
          solver='cd', 
          beta_loss='frobenius', 
#           Tolerance of the stopping condition
          tol=0.01, #поменяли
          max_iter=100, 
          random_state=1, 
#           Constant that multiplies the regularization terms
          alpha=0.0, 
#           For l1_ratio = 0 the penalty is 
#           an elementwise L2 penalty (aka Frobenius Norm). 
          l1_ratio=0.0,  
#           If true, randomize the order of coordinates in the CD solver.
          shuffle=False)

In [13]:
# %%time

data['nmf_cosine_similarity'] = [cosine_similarity(
    np.atleast_2d(nmf.fit_transform(
        tv.transform(data['text_1_norm']))[i]), 
    np.atleast_2d(nmf.fit_transform(
        tv.transform(data['text_2_norm']))[i]))[0,0] 
                                 for i in range(len(data))]

In [21]:
X5 = data[data.columns[-5:]].to_numpy()

In [22]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=8, #увеличили
                            min_samples_leaf=15,
                            class_weight='balanced')
clf.fit(X5, data['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
%%time

for score_type in 'weighted micro macro'.split():
    print ('Mean validation {} F1-score:'.format(score_type), 
        round(
            float(np.mean(cross_val_score(
            estimator=clf,
            X=X5,
            y=data['label'],   
            scoring='f1_{}'.format(score_type),
            cv=15, 
            n_jobs=-1))),
            5))

# Micro f1 на близостях в первый раз
# Mean validation micro F1-score: 0.54904

Mean validation weighted F1-score: 0.55242
Mean validation micro F1-score: 0.5572
Mean validation macro F1-score: 0.54995
Wall time: 40.6 s


Незначительные улучшения метрики.